## Librairies importation

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import torch
from torchvision import models
import torch
from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets, models
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
import os
import numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.nn import functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import torch
from PIL import Image
from torchvision import transforms
from pylab import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
## googlenet model downloading
googlenet= models.googlenet(pretrained=True)

In [ ]:
# googlenet model evaluation
googlenet.eval()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
im = Image.open("/content/drive/MyDrive/BootCamp_Week_1/70bfca4555cca92e (2).jpg")
im 
# !wget https://github.com/benjaminbenteke/AMMI_BootCamp_Week_1/blob/28425f17b18823beb7431749bb499bdde355dfb2/image.jpg

In [ ]:
img= cv2.imread("/content/drive/MyDrive/BootCamp_Week_1/70bfca4555cca92e (2).jpg")
img.shape

In [ ]:
def pre_processing(obs, cuda):
    mean = np.array([0.485, 0.456, 0.406]).reshape([1, 1, 3])
    std = np.array([0.229, 0.224, 0.225]).reshape([1, 1, 3])
    obs = obs / 255
    obs = (obs - mean) / std
    # obs = np.transpose(obs, (2, 0, 1))
    # obs = np.expand_dims(obs, 0)
    obs = np.array(obs)
    if cuda:
        torch_device = torch.device('cuda:0')
    return obs

In [ ]:
img= cv2.imread('/content/drive/MyDrive/BootCamp_Week_1/70bfca4555cca92e (2).jpg')
img= pre_processing(img, False)
img= np.transpose(img, (2,0,1))
img= torch.tensor(img)
img= np.expand_dims(img, 0)
img= torch.tensor(img,dtype=torch.float)
img.shape

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [ ]:
with open('imagenet_classes.txt') as f:
  categories = [line.strip() for line in f.readlines()]

In [ ]:
with torch.no_grad():
    output = googlenet(img)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

In [ ]:
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

im

In [ ]:
categories.index('fireboat') ## Target index for the image.

# Integrated gradient methods
* Idea behind Path;
* Interpolated image;
* Gradient;
* Integrated Gradinet.

In [ ]:
# Image importation

img= cv2.imread('/content/drive/MyDrive/BootCamp_Week_1/70bfca4555cca92e (2).jpg')
img= pre_processing(img, False)

In [ ]:
img.shape

In [ ]:
# Interpolate images function

def intepolate_img(input,baseline, alphas):
  alphas_x = alphas[:, tf.newaxis, tf.newaxis, tf.newaxis]
  alphas_x=tf.make_ndarray(tf.make_tensor_proto(alphas_x))
  alphas_x=torch.tensor(alphas_x)

  baseline_x= torch.unsqueeze(baseline, 0)
  input_x= np.expand_dims(input, 0)
  input_x = torch.tensor(input_x,dtype=float)
  x_x_prime = input_x - baseline_x

  images = baseline_x + alphas_x*x_x_prime

  return images

In [ ]:
# baseline creation
baseline = torch.zeros(size=(224,224,3))
baseline.shape

In [ ]:
figure(1)
imshow(baseline, interpolation='nearest')

In [ ]:
img.shape

In [ ]:
m_steps=50
alphas = tf.linspace(start=0.0, stop=1.0, num=m_steps+1)

In [ ]:
# Interpolation image computatation
interpolated_images= intepolate_img(input=img, baseline=baseline,alphas=alphas) # This the path (all 51 images from baseline to the original image, include the baseline)

In [ ]:
interpolated_images.shape

<!-- <img src='/content/drive/MyDrive/BootCamp_Week_1/Integrated_gradient.png'> -->

In [ ]:
fig = plt.figure(figsize=(20, 20))

i = 0
for alpha, image in zip(alphas[0::10], interpolated_images[0::10]):
  i += 1
  plt.subplot(1, len(alphas[0::10]), i)
  plt.title(f'alpha: {alpha:.1f}')
  plt.imshow(image)
  plt.axis('off')

plt.tight_layout()

In [ ]:
Image.open('/content/drive/MyDrive/BootCamp_Week_1/Integrated_gradient.png')

# Explanations of the Paper. Brief
* Interpolated images are image of the path, form baseline to original. They are obtained by increasing alpha (brightness), and for each interplated image, we make a prediction.
* The value of alpha give us the number of interporlated images ([alpha,224,224,3]) because for each alpha we have an associated image.
* The set of all interpolated images is called a Path. Path (all image from baseline to original image), in our case the path is a straight line. Or a Path is an array that contains all interpolated images.
* We'll compute the gradient for interpolated_images and we aggregate them. This aggregated gradient is called, Integrated Gradient. # See the figure below

In [ ]:
# Prediction and Gradient 
def output_and_gradient_computation(inputs, model, target_label_idx, cuda=False):
  
    gradients = []

    for input in inputs:

      input= np.transpose(input, (2,0,1))
      input= torch.tensor(input,dtype=torch.float)
      input= input.unsqueeze(0)
      input.requires_grad = True
      output = model(input)
      output = F.softmax(output, dim=1)


      if target_label_idx is None:
          target_label_idx = torch.argmax(output, 1).item()
      index = np.ones((output.size()[0], 1)) * target_label_idx
      index = torch.tensor(index, dtype=torch.int64)
      if cuda:
          index = index.cuda()
      output = output.gather(1, index)

      model.zero_grad()
      output.backward()

      gradient = input.grad.detach().cpu().numpy()[0]
      gradients.append(gradient)
    gradients = np.array(gradients)
    return gradients, output, target_label_idx

In [ ]:
grad, output, _= output_and_gradient_computation(interpolated_images, googlenet, 554, False)

In [ ]:
output

In [ ]:
grad.shape

In [ ]:
## Integrated gradient implementation

def integrated_gradients(inputs, model, target_label_idx, res, baseline, steps=50, cuda=False):
  if baseline is None:
        baseline = 0 * inputs 
  baseline= np.array(baseline)
  avg_grads = np.average(res[:-1], axis=0)
  avg_grads = np.transpose(avg_grads, (1, 2, 0))
  delta_X = (pre_processing(inputs, cuda) - pre_processing(baseline, cuda))
  integrated_grad = delta_X * avg_grads

  return integrated_grad

In [ ]:
 ig= integrated_gradients(img, googlenet, 554, grad, baseline, 50, False)

In [ ]:
ig.shape # must be the same shape as the original image

In [ ]:
## Visualizations
def plot_img_attributions(baseline, image, target_class_idx, ig, m_steps=50, cmap=None, overlay_alpha=0.4):

  attribution_mask = tf.reduce_sum(tf.math.abs(ig), axis=-1)

  fig, axs = plt.subplots(nrows=2, ncols=2, squeeze=False, figsize=(8, 8))

  axs[0, 0].set_title('Baseline image')
  axs[0, 0].imshow(baseline)
  axs[0, 0].axis('off')

  axs[0, 1].set_title('Original image')
  axs[0, 1].imshow(image)
  axs[0, 1].axis('off')

  axs[1, 0].set_title('Attribution mask')
  axs[1, 0].imshow(attribution_mask, cmap=cmap)
  axs[1, 0].axis('off')

  axs[1, 1].set_title('Overlay')
  axs[1, 1].imshow(attribution_mask, cmap=cmap)
  axs[1, 1].imshow(image, alpha=overlay_alpha)
  axs[1, 1].axis('off')

  plt.tight_layout()
  return fig

In [ ]:
plot_img_attributions(baseline, im, 554,ig, m_steps=50, cmap=plt.cm.inferno, overlay_alpha=0.4)

In [ ]:
grad.shape

# Average gradient at the image

In [ ]:
avg_grads = np.average(grad[:-1], axis=0)
avg_grads= np.transpose(avg_grads,(2,1,0))

In [ ]:
plot_img_attributions(baseline, im, 554,avg_grads, m_steps=50, cmap=plt.cm.inferno, overlay_alpha=0.4)

# End of the code. (these codes must be improved)

In [ ]:
from PIL import Image
im= Image.open("/content/drive/MyDrive/BootCamp_Week_1/c0a9ce885a9c26bc.jpg")
im 

In [ ]:
img= cv2.imread('/content/drive/MyDrive/BootCamp_Week_1/c0a9ce885a9c26bc.jpg')
img= pre_processing(img, False)

In [ ]:
img.shape

In [ ]:
img= np.transpose(img, (2,0,1))
img= torch.tensor(img)
img= np.expand_dims(img, 0)
img= torch.tensor(img,dtype=torch.float)
img.shape

In [ ]:
with torch.no_grad():
    output = googlenet(img)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

In [ ]:
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

im

In [ ]:
categories.index('viaduct') ## Target index for the image.

In [ ]:
# Apply IG

img= cv2.imread('/content/drive/MyDrive/BootCamp_Week_1/c0a9ce885a9c26bc.jpg')
img= pre_processing(img, False)

In [ ]:
# Interpolation image computatation
interpolated_images= intepolate_img(input=img, baseline=baseline,alphas=alphas) 

In [ ]:
fig = plt.figure(figsize=(20, 20))

i = 0
for alpha, image in zip(alphas[0::10], interpolated_images[0::10]):
  i += 1
  plt.subplot(1, len(alphas[0::10]), i)
  plt.title(f'alpha: {alpha:.1f}')
  plt.imshow(image)
  plt.axis('off')

plt.tight_layout()

In [ ]:
grad, output, _= output_and_gradient_computation(interpolated_images, googlenet, 554, False)

In [ ]:
 ig= integrated_gradients(img, googlenet, 554, grad, baseline, 50, False)

In [ ]:
plot_img_attributions(baseline, im, 554,ig, m_steps=50, cmap=plt.cm.inferno, overlay_alpha=0.4)

In [ ]:
avg_grads = np.average(grad[:-1], axis=0)
avg_grads= np.transpose(avg_grads,(2,1,0))

In [ ]:
plot_img_attributions(baseline, im, 554,avg_grads, m_steps=50, cmap=plt.cm.inferno, overlay_alpha=0.4)

In [ ]:
## 